- https://www.cnblogs.com/bytehandler/p/17635933.html

- `export NCCL_P2P_DISABLE=1`：disable direct GPU-to-GPU (P2P) communication.
    - 禁用点对点通信，点对点通信是一种高效的数据传输方式，允许直接在 GPU 之间传输数据，绕过主机内存（CPU 内存）。但在某些情况下，点对点通信可能会导致兼容性或性能问题。
    - Using RTX 4000 series which doesn't support faster communication speedups. Ensuring P2P and IB communications are disabled.

In [1]:
from IPython.display import Image

## pytorch 分布式相关api

- `torch.distributed.init_process_group()` ，初始化进程组。
- `torch.distributed.get_rank()`，可以获得当前进程的 rank；
- `torch.distributed.get_world_size()`，可以获得进程组的进程数量。
- `torch.distributed.barrier()`，同步进程组内的所有进程，阻塞所有进程直到所有进程都执行到操作。

## 集合通信

NCCL支持集合通信操作（Collective Operations）：

- `Broadcast`，进程组内的一个进程将Tensor广播给其他进程。
- `AllReduce`，进程组内所有进程进行规约操作，最终所有进程得到统一的Tensor。
- `ReduceScatter`，进程组内所有进程先进行reduce操作，再进行scatter操作，每个进程得到Tensor的一部分。
- `AllGather`，进程组内所有进程的Tensor聚合成一个Tensor列表，并且最终所有进程都有一个Tensor列表副本。

In [3]:
Image(url='https://pytorch.org/tutorials/_images/scatter.png', width=400)

```
def dist_scatter():
    dist.barrier()

    rank = dist.get_rank()
    world_size = dist.get_world_size()

    tensor = torch.zeros(world_size)
    before_tensor = tensor.clone()
    if dist.get_rank() == 0:
        # Assumes world_size of 2.
        # Only tensors, all of which must be the same size.
        t_ones = torch.ones(world_size)
        t_fives = torch.ones(world_size) * 5
        scatter_list = [t_ones, t_fives]
    else:
        scatter_list = None
    dist.scatter(tensor, scatter_list, src=0)
    logging.info(f"broadcast, rank: {rank}, before scatter: {repr(before_tensor)} after scatter: {repr(tensor)}")
    dist.barrier()
```